In [ ]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
import os
from load_dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("API KEY VARAIBLE EXIST")
else:
   raise ValueError("OPENAI_API_KEY NOT FOUND") 

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "user", "content": "Hello!"}
    ]
)

print(response.choices[0].message.content)

Hi there! How can I help you today — a question, writing or coding help, planning something, or something else?


## LLM CALL 


In [ ]:
from langchain_openai import ChatOpenAI

import os
from load_dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("API KEY VARAIBLE EXIST")
else:
   raise ValueError("OPENAI_API_KEY NOT FOUND") 
llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

llm_openai.invoke("Bro, tell me a fun fact").content

"Bro — fun fact: wombats poop in perfect little cubes.  \n\nTheir intestines shape the poop into cubes so it doesn't roll away, which helps them mark territory and communicate. Wildly practical and oddly adorable. Want another one?"

### USING init_chat_model  a factory function that automatically initializes the correct chat model class based on the provider you specify, without needing to import provider-specific classes

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI
import os
from load_dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("API KEY VARAIBLE EXIST")
else:
   raise ValueError("OPENAI_API_KEY NOT FOUND") 
   
llm = init_chat_model("gpt-5-mini", model_provider="openai")

llm.invoke("Bro, tell me a fun fact").content


'Bro — fun fact: wombats poop in perfect little cubes. They do it to stack the piles as territorial/attracting markers, and their uniquely-shaped, very slow-moving intestines are what mold the poop into cubes. Want another weird one?'

# MESSAGES

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
import os
from load_dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("API KEY VARAIBLE EXIST")
else:
   raise ValueError("OPENAI_API_KEY NOT FOUND") 

llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

my_message = [
    HumanMessage(content="Bro, tell me a fun fact?"),
    SystemMessage(content="You are a gen-z assistance. who always answer in afun way")
]

llm_openai.invoke(my_message).content

API KEY VARAIBLE EXIST


'Bro — fun fact: bananas are technically berries, but strawberries aren’t. Wild, right? Botanically a “berry” comes from one ovary (bananas, grapes, even tomatoes), while strawberries are aggregate fruits with seeds on the outside. Mind. Blown. 🤯🍌'

In [ ]:
from langchain_openai import ChatOpenAI
import os
from load_dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("API KEY VARAIBLE EXIST")
else
   raise ValueError("OPENAI_API_KEY NOT FOUND")    

API KEY VARAIBLE EXIST


# PROMPT

In [23]:
from langchain_core.prompts import PromptTemplate

user_input =input("Enter a topic for fun fact : ")

dynamic_prompt = PromptTemplate.from_template("Write a fun fact about {topic}")

ready_prompt = dynamic_prompt.invoke({"topic" : user_input})

llm_openai.invoke(ready_prompt).content

"Fun fact: Domestic cats can't taste sweetness — they lack a functional sweet-taste receptor (a mutated Tas1r2 gene), so sugary foods don't register as sweet to them."

## USING A ChatPromptTemplate

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini")

template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert {language} code reviewer. Provide constructive feedback."),
    ("human", "Review this code:\n{code}")
])

prompt = template.invoke({
    "language": "Java",
    "code": "public class Test { }"
})

llm.invoke(prompt).content


'Summary\n- The code compiles but is an empty, public top-level class: public class Test { }.\n- As-is it has no behavior or documentation. That may be fine temporarily, but in real code it should be improved for clarity, purpose, packaging and style.\n\nConcrete suggestions\n\n1) Give the class a meaningful name\n- "Test" is generic and likely to collide with testing frameworks and other classes. Choose a name that describes the class responsibility (e.g., UserService, Customer, StringUtils).\n\n2) Put classes in a package\n- Avoid the default package. Add a package declaration (e.g., package com.example.foo;). This prevents name clashes and is required for production builds and many tools.\n\n3) Add Javadoc\n- At minimum document the class responsibility and any invariants or usage notes:\n  /**\n   * Brief description of what this class represents or does.\n   */\n\n4) Don’t leave an empty public class\n- If the class is intended to be a marker type use an interface or annotation, o

In [19]:
my_message = [
    HumanMessage(content="Bro, tell me a fun fact about honey?"),
    SystemMessage(content="You are a gen-z assistance. who always answer in afun way")
]
llm_openai.invoke(my_message).content

'Bro — fun fact: honey basically never spoils. Archaeologists have found 3,000‑year‑old jars of honey in Egyptian tombs that were still edible, thanks to honey’s low water content, acidity, and bee enzymes that make it inhospitable to bacteria.'

PYDANTIC AND STURCTURE OUTPUT


In [40]:
from pydantic import BaseModel, field_validator, Field
from datetime import date, timedelta
from typing import Literal

class LAMISPatient(BaseModel):
    """Production-ready patient model"""
    
    patient_id: str = Field(pattern=r"^LAM\d+$")
    name: str
    date_of_birth: date
    cd4_count: int
    viral_load: int
    regimen: str
    adherence: Literal["Good", "Fair", "Poor"]
    last_visit: date
    
    # VALIDATORS - Data quality checks (V2 syntax)
    
    @field_validator('cd4_count')
    @classmethod
    def validate_cd4(cls, v: int) -> int:
        """Check CD4 range and warn"""
        if v < 0 or v > 2000:
            raise ValueError(f"Invalid CD4: {v}")
        if v < 200:
            print(f"🚨 CRITICAL CD4: {v} - Immediate action required")
        return v
    
    @field_validator('last_visit')
    @classmethod
    def check_overdue(cls, v: date) -> date:
        """Warn if visit is overdue"""
        days_since = (date.today() - v).days
        if days_since > 90:
            print(f"⚠️ Patient overdue by {days_since} days")
        return v
    
    # PROPERTIES - Computed business logic
    
    @property
    def age(self) -> int:
        """Current age"""
        return (date.today() - self.date_of_birth).days // 365
    
    @property
    def is_pediatric(self) -> bool:
        """Check if pediatric patient"""
        return self.age < 18
    
    @property
    def treatment_outcome(self) -> str:
        """WHO treatment outcome classification"""
        if self.viral_load < 1000 and self.cd4_count > 350:
            return "Virological Success"
        elif self.viral_load >= 1000:
            return "Virological Failure"
        else:
            return "Immunological Failure"
    
    @property
    def next_visit_due(self) -> date:
        """Calculate next visit date"""
        if self.adherence == "Poor":
            return self.last_visit + timedelta(days=30)
        else:
            return self.last_visit + timedelta(days=90)
    
    @property
    def clinical_alert(self) -> str | None:
        """Generate clinical alerts"""
        alerts = []
        
        if self.cd4_count < 200:
            alerts.append("Critical CD4 count")
        
        if self.viral_load > 1000 and self.adherence == "Good":
            alerts.append("Treatment failure despite good adherence")
        
        days_overdue = (date.today() - self.last_visit).days - 90
        if days_overdue > 0:
            alerts.append(f"Overdue by {days_overdue} days")
        
        return "; ".join(alerts) if alerts else None


# Usage example
if __name__ == "__main__":
    patient = LAMISPatient(
        patient_id="LAM12345",
        name="John Doe",
        date_of_birth=date(1980, 5, 15),
        cd4_count=180,
        viral_load=75000,
        regimen="TDF/3TC/EFV",
        adherence="Good",
        last_visit=date(2025, 11, 1)
    )

    # Validators ran automatically during creation
    print(f"Age: {patient.age}")
    print(f"Outcome: {patient.treatment_outcome}")
    print(f"Next Visit: {patient.next_visit_due}")
    print(f"Alerts: {patient.clinical_alert}")

🚨 CRITICAL CD4: 180 - Immediate action required
⚠️ Patient overdue by 93 days
Age: 45
Outcome: Virological Failure
Next Visit: 2026-01-30
Alerts: Critical CD4 count; Treatment failure despite good adherence; Overdue by 3 days
